In [ ]:
##Python for String Processing (Much Faster)
#Instead of using grep inside a Bash loop, use Python, which handles string operations much more efficiently. 
#Modify your script to call a Python script for counting occurrences. 
#Call the python script: count_telomere_variants.py

import sys
from collections import defaultdict

# Get input filenames
bam_reads_file = sys.argv[1]   # FASTQ file from BAM
repeats_file = sys.argv[2]     # File with telomere repeats
output_file = sys.argv[3]      # Output results file

# Load telomere repeat variants into a list
with open(repeats_file, "r") as f:
    repeats = [line.strip() for line in f if line.strip()]

# Open the output file for writing
with open(output_file, "w") as out:
    out.write("Read_ID\tTotal_TVR_Count\tTVR_Type\n")

    # Read through the FASTQ file, processing every 4th line as a sequence
    with open(bam_reads_file, "r") as f:
        lines = f.readlines()
        for i in range(0, len(lines), 4):
            read_id = lines[i].strip()[1:]  # Remove '@' from header
            sequence = lines[i + 1].strip()  # Read the sequence
            
            total_tvr_count = 0
            tvr_counts = defaultdict(int)

            # Count occurrences of each repeat in the sequence
            for repeat in repeats:
                count = sequence.count(repeat)
                if count > 0:
                    total_tvr_count += count
                    tvr_counts[repeat] += count
            
            # If repeats were found, write to output
            if total_tvr_count > 0:
                tvr_types = ", ".join(f"{k} ({v})" for k, v in tvr_counts.items())
                out.write(f"{read_id}\t{total_tvr_count}\t{tvr_types}\n")


In [ ]:
## python_count_telomere_variants_table.sh:


#!/bin/bash
#SBATCH --mem=32G
#SBATCH -p gpu
#SBATCH --gres=gpu:A100:1
#SBATCH -c 8
#SBATCH -t 300

# Ensure required variables are set
if [[ -z "$BAM_FILE" || -z "$REPEATS_FILE" || -z "$OUTPUT_FILE" ]]; then
    echo "Error: BAM_FILE, REPEATS_FILE, or OUTPUT_FILE is not set."
    exit 1
fi

# Convert BAM to FASTQ
samtools fastq "$BAM_FILE" > reads.fastq

# Run Python script
python3 count_telomere_variants.py reads.fastq "$REPEATS_FILE" "$OUTPUT_FILE"

echo "Counting complete. Results saved to $OUTPUT_FILE."

In [ ]:
##Submit your Slurm job, request multiple CPUs
export BAM_FILE="/storage/scratch01/groups/bu/teloseq/TVR/path/to/filtered_aligned/.bam"
export REPEATS_FILE="/storage/scratch01/groups/bu/teloseq/TVR/telomere_repeats.txt"
export OUTPUT_FILE="/storage/scratch01/groups/bu/teloseq/TVR/telomere_variants_table.txt"
sbatch -c 8 --gres=gpu:A100:1 python_count_telomere_variants_table.sh